Учебное задание из курса МФТИ "Машинное обучение и анализ данных" на Courcera.  
Легенда такова:
Компания Carnival Cruise Line хочет разместить баннеры максимально близко к своим офисам. У компании офисы разбросанны по всему миру. Баннеры должны находить в районах с большой проходимостью. Колличество баннеров ограниченно 20. 

Порядок действий следующий:
1. Получить информацию о посещении мест (файл checkins.dat содержит геокоординаты чек-инов). https://archive.org/details/201309_foursquare_dataset_umn
2. Сгруппировать по кластерам. Каждый кластер содержит чек-ины в радиусе 5 километров.
3. Найти центр 20 кластеров (кластер должен содержать более 15 точек), которые максимально близки к каждому офису.

In [15]:
import pandas as pd
import numpy as np

Загружаем данные из файла __checkins.dat__. Этот файл содержит информацию о чек-инах с разделителем __'|'__.

In [3]:
data=pd.read_csv('checkins.dat', delimiter='|')

Посмотрим на первые 5 строк датасета, а так же узнаем его размер.

In [4]:
data.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984301,2041916,5222,,,2012-04-21 17:39:01
1,984222,15824,5222,38.8951118,-77.0363658,2012-04-21 17:43:47
2,984315,1764391,5222,,,2012-04-21 17:37:18
3,984234,44652,5222,33.800745,-84.41052,2012-04-21 17:43:43
4,984249,2146840,5222,,,2012-04-21 17:42:58


In [5]:
data.shape

(1021966, 6)

Итак, мы имеем более миллиона строк, но не все они содржат полезную информацию. __От строк с пропусками координат необходимо избавиться.__
К сожалению, данные в файле разделялись не только разделителем, но и имели форматирование пробелами. Из-за этого мы имеем _не правильное название столбцов, а так же лишние пробелы в данных_. Эту проблему нужно решить.

Посмотрим на список столбцов.

In [6]:
data.columns

Index(['   id    ', ' user_id ', ' venue_id ', '     latitude      ',
       '     longitude     ', '     created_at      '],
      dtype='object')

Заменим их на нормальный список.

In [7]:
data.columns=['id','user_id','venue_id','latitude','longitude','created_at']

Теперь разберёмся с лишними пробелами.

In [8]:
data.latitude[0]

'                   '

In [9]:
data.latitude=data.latitude.str.strip()
data.longitude=data.longitude.str.strip()

In [10]:
data.latitude[0]

''

Отлично. Удалим строки, которые не содержат координат.

In [11]:
data=data[(data.latitude!='') & (data.longitude!='')]

In [12]:
data.head()

,id,user_id,venue_id,latitude,longitude,created_at
1,984222,15824,5222,38.8951118,-77.0363658,2012-04-21 17:43:47
3,984234,44652,5222,33.800745,-84.41052,2012-04-21 17:43:43
7,984291,105054,5222,45.5234515,-122.6762071,2012-04-21 17:39:22
9,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
10,984232,93870,380645,33.4483771,-112.0740373,2012-04-21 17:38:18


In [13]:
data.shape

(396634, 6)

In [14]:
X=data.drop(['id','user_id','venue_id','created_at'],axis=1)

У нас осталось около 400 тысяч строк с координатами.

Необходимо все координаты объеденить в кластеры и получить центры больших кластеров (более 15 элементов)

Будем использовть кластеризатор __MeanShift__ из библиотеки __sklearn.cluster__  
Для увеличения скорости я буду работаь олько с первыми 100000 строками.

In [18]:
%%time
from sklearn.cluster import MeanShift
clust=MeanShift(bandwidth=0.1).fit(X[:100000])

CPU times: user 2min 55s, sys: 479 ms, total: 2min 55s
Wall time: 2min 56s


In [19]:
len(clust.cluster_centers_)

3230

Мы получили 3230 кластеров, но нам интересны кластеры с количеством точек больше 15

In [20]:
unique, counts = np.unique(clust.labels_, return_counts=True)

In [21]:
cents=[]
for x in unique:
    cents.append(clust.cluster_centers_[x])

In [22]:
df = pd.DataFrame(data={'value':unique, 'counts':counts, 'centers':cents})

In [23]:
len(df)

3230

In [24]:
df=df[df.counts>15]

In [25]:
len(df)

592

Итого у нас получилось 592 больших кластера

Нам нужно найти 20 центров которые максимально близки к точкам с координатами:
    
* 33.751277, -118.188740 (Los Angeles)
* 25.867736, -80.324116 (Miami)
* 51.503016, -0.075479 (London)
* 52.378894, 4.885084 (Amsterdam)
* 39.366487, 117.036146 (Beijing)
* -33.868457, 151.205134 (Sydney)



Как расположены офисы на карте.  
https://www.mapcustomizer.com/map/simple_map

In [26]:
def length(a,b):
    return np.sqrt(((a[0]-b[0])**2)+((a[1]-b[1])**2))

In [27]:
city=[[33.751277, -118.188740],[25.867736, -80.324116],[51.503016, -0.075479],
      [52.378894, 4.885084],[39.366487, 117.036146],[-33.868457, 151.205134]]

In [28]:
centers=[]
lengths=[]
for city_c in city:
    for center in df.centers:
        centers.append(center)
        lengths.append(length(city_c,center))

In [29]:
centers_len = pd.DataFrame(data={'centers':centers, 'lengths':lengths})

In [30]:
k=centers_len.sort_values(['lengths'])[:20]

In [31]:
print(k.centers)

3363      [-33.86063042857143, 151.20477592857145]
2148        [52.37296399032261, 4.892317222580647]
1004        [25.8456722642857, -80.31889059642857]
1242    [51.502991260887086, -0.12553728870967767]
51         [33.8098779552631, -118.14892380690813]
621          [25.7858124199675, -80.2179380368254]
758       [25.705349721052592, -80.28342873815798]
684       [26.010098249285683, -80.19999058571432]
87         [33.8883253427586, -118.04892817172427]
42        [33.87298601157018, -118.36209114655645]
293       [33.97257482142858, -118.16837066666663]
907       [26.138843786842077, -80.33434683684207]
119       [33.98393587403844, -118.00740497307689]
647        [26.120862658633104, -80.1589066802157]
27          [33.81730643390889, -117.891249170958]
11        [34.06039755458241, -118.24870902659876]
32        [33.67430265976576, -117.85878926777275]
752        [26.20058464102565, -80.25071612564099]
17        [34.03548695312116, -118.43899771946148]
47        [34.13146014917951, -

Как расположены баннеры относительно офисов.
https://www.mapcustomizer.com/map/simple_clusters  
Получилось, что в нескольких городах много баннеров, а некоторых ниодного. Возможно, кластеризация на всей выборке поможет решиь эту проблему.